In [3]:
# загружаем зависимости
!pip install --upgrade pip
!pip install sympy
!pip install numpy
!pip install scipy
!pip install tqdm
!pip install matplotlib

In [2]:
# библиотека для символьных вычислений
import numpy as np
import scipy as sp
from sympy import Rational, sqrt, I, symbols, simplify, Eq, latex
from itertools import product

In [17]:
# задаём константы

eps = -Rational(1, 2) + sqrt(3)/2*I

# t_{ij}^{(m)} <=> t[m][i][j]
t = np.array([
    [
        [eps, 0, 0],
        [0, 1, 0],
        [0, 0, eps**2],
    ],
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ],
    [
        [eps**2, 0, 0],
        [0, 1, 0],
        [0, 0, eps],
    ],
    [
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 0],
    ],
    [
        [0, -1, 1],
        [1, 0, -1],
        [-1, 1, 0],
    ],
    [
        [0, eps, 1],
        [eps, 0, eps**2],
        [1, eps**2, 0],
    ],
    [
        [0, eps**2, 1],
        [eps**2, 0, eps],
        [1, eps, 0],
    ],
    [
        [0, -eps**2, 1],
        [eps**2, 0, -eps],
        [-1, eps, 0],
    ],
    [
        [0, -eps, 1],
        [eps, 0, -eps**2],
        [-1, eps**2, 0],
    ],
])

t[4] *= I / sqrt(3)
t[7] *= I / sqrt(3)
t[8] *= I / sqrt(3)

# h_{d,m}
h = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1],  # phi1
    [1, 1, 1, 1, -1, 1, 1, -1, -1],  # phi2
    [eps, 1, eps**2, 1, -1, eps**2, eps, -eps, -eps**2],  # phi3
    [eps**2, 1, eps, 1, -1, eps, eps**2, -eps**2, -eps],  # phi4
    [eps**2, 1, eps, 1, 1, eps, eps**2, eps**2, eps],  # phi5
    [eps, 1, eps**2, 1, 1, eps**2, eps, eps, eps**2],  # phi6
])

In [4]:
# задаём перестановки

# psi: (0, 1, 2, 3, 4, 5, 6, 7, 8)
def psi_even(m):
    return m

# psi: (2, 1, 0, 3, 4, 6, 5, 8, 7)


def psi_odd(m):
    match m:
        case 0:
            return 2
        case 2:
            return 0
        case 5:
            return 6
        case 6:
            return 5
        case 7:
            return 8
        case 8:
            return 7
        case _:
            return m


psi = np.array([
    psi_even,
    psi_odd,
    psi_odd,
    psi_odd,
    psi_even,
    psi_even,
])

In [19]:
# 54 переменные искомой системы
X = symbols("x:9")
Y = symbols("y:9")
Z = symbols("z:9")
A = symbols("a:9")
B = symbols("b:9")
C = symbols("c:9")

In [20]:
# phi_d(x_m)
def phi(d, X, m):
    return h[d-1][m]*X[psi[d-1](m)]


# правая часть уравнения из исходной системы
def f(i, j, k, l, r, s):
    if (i == j and k == l and r == s) and not (i == j == k == l == r == s):
        return 1
    if (j == k and l == r and s == i) and not (i == j == k == l == r == s):
        return -1
    return 0

In [22]:
# левая часть уравнения
def generate_left(m, p, q):
    left = 2*(X[m]*Y[p]*Z[q] + X[p]*Y[q]*Z[m] + X[q]*Y[m]*Z[p] +
              phi(2, X, m)*phi(2, Y, p)*phi(2, Z, q) +
              phi(2, X, p)*phi(2, Y, q)*phi(2, Z, m) +
              phi(2, X, q)*phi(2, Y, m)*phi(2, Z, p))
    for d in range(3, 7):
        left += phi(d, A, m)*phi(d, B, p)*phi(d, C, q) +\
            phi(d, A, p)*phi(d, B, q)*phi(d, C, m) +\
            phi(d, A, q)*phi(d, B, m)*phi(d, C, p)
    return simplify(left)


# правая часть уравнения
def generate_right(m, p, q):
    right = 0
    for i, j, k, l, r, s in product(range(3), repeat=6):
        right += t[m][i][j]*t[p][k][l]*t[q][r][s]*f(i, j, k, l, r, s)
    return simplify(right)


def generate_equation(m, p, q):
    left = generate_left(m, p, q)
    right = generate_right(m, p, q)
    return Eq(left, right)


equations = []
is_counted = np.zeros((9, 9, 9), bool)
with open("equations.tex", "w") as equations_file:
    for m, p, q in product(range(9), repeat=3):
        if is_counted[m][p][q]:
            continue

        # поскольку уравнения для шестёрок (m, p, q), (p, q, m), (q, m, p),
        # (phi_2(m), phi_2(p), phi_2(q)), (phi_2(p), phi_2(q), phi_2(m)), (phi_2(q), phi_2(m), phi_2(p))
        # оказываются идентичными
        is_counted[m][p][q] = True
        is_counted[p][q][m] = True
        is_counted[q][m][p] = True
        is_counted[psi[1](m)][psi[1](p)][psi[1](q)] = True
        is_counted[psi[1](p)][psi[1](q)][psi[1](m)] = True
        is_counted[psi[1](q)][psi[1](m)][psi[1](p)] = True

        equation = generate_equation(m, p, q)
        if equation == True:
            continue
        equations.append(equation)
        equations_file.write(f"\\[ {latex(equation)} \\]\n")

len(equations)

125

## Compressed form of equations

In [8]:
# для уравнений в сжатом виде
XYZ = symbols("xyz:9:9:9")
ABC = symbols("abc:9:9:9")

In [9]:
def get_index(m, p, q):
    return 81*m + 9*q + p


# phi_d(p_{mpq})
def phi(d, XYZ, m, p, q):
    return h[d-1][m]*h[d-1][p]*h[d-1][q]*XYZ[get_index(psi[d-1](m), psi[d-1](p), psi[d-1](q))]

In [14]:
# левая часть уравнения
def generate_left(m, p, q):
    left = 2*(XYZ[get_index(m, p, q)] + phi(2, XYZ, m, p, q))
    for d in range(3, 7):
        left += phi(d, ABC, m, p, q)
    return simplify(left)


compressed_equations = []
is_counted = np.zeros((9, 9, 9), bool)
with open("compressed_equations.tex", "w") as equations_file:
    for m, p, q in product(range(9), repeat=3):
        if is_counted[m][p][q]:
            continue

        # поскольку уравнения для шестёрок (m, p, q), (p, q, m), (q, m, p),
        # (phi_2(m), phi_2(p), phi_2(q)), (phi_2(p), phi_2(q), phi_2(m)), (phi_2(q), phi_2(m), phi_2(p))
        # оказываются идентичными
        is_counted[m][p][q] = True
        is_counted[p][q][m] = True
        is_counted[q][m][p] = True
        is_counted[psi[1](m)][psi[1](p)][psi[1](q)] = True
        is_counted[psi[1](p)][psi[1](q)][psi[1](m)] = True
        is_counted[psi[1](q)][psi[1](m)][psi[1](p)] = True
        
        equation = generate_equation(m, p, q)
        if equation == True:
            continue

        compressed_equations.append(equation)
        equations_file.write(f"\\[ {latex(equation)} \\]\n")

Eq(2*abc000 + 2*abc222 + 2*xyz000 + 2*xyz222, -3)


## Practice

In [23]:
# Переменные, относительно которых решаем систему.
vars = [X[7], Y[7], Z[7], A[7], B[7], C[7], X[8], Y[8], Z[8], A[8], B[8], C[8]]

def contains(equation, vars):
    return set(vars) & equation.free_symbols

In [47]:
# Подставляем решенеие, найденное Александром для переменных групп (0) (2) | (5) (6) (3) | (4).
# Для всех переменных группы (1) задаём значение 1.

raw_values = '''
-0.0020183703472882 -0.00161592947542374 | -0.00323185996072685 0.0020223128605026  0.498107722299555 | -87.0299370852072 -55.4509377762183 0.500631616538801
-40784.8393796599 82.5010995386101 | -82.3410303678316 -81569.7252586689  -0.512256798857592 | -29.0282928936237 -95.1678278154882 -0.000467783310792664
-0.00202202644535711 0.00162001578179385 | 0.00324003082356708 0.00202592619390263  0.501896239699552 | -76.4796388644552 24.8201036832838 -0.49936879502249
-0.00201837039476224 -0.0016159285515645 | -0.00323185933429619 0.00202231296512867  0.498107723153734 | 54.7408764168019 -34.2056358172428 0.50063162893345
-81569.6794472786 164.999101204865 | -164.679522002895 -163139.449136568  -1.48774478355516 | -44.6036768345083 85.387523402247 0.00040283837741723
-0.00202202636916792 0.00162001512828787 | 0.00324003336320042 0.00202592615782511  0.501896239139354 | 71.484617908198 10.711370234807 -0.499368787583764
'''

vars_order = [X, Y, Z, A, B, C]

subs = []

lines = raw_values.split('\n')[1:-1]
for i, line in enumerate(lines):
    var = vars_order[i]
    values = line.replace('|', '').split()
    indeces_order = {0: 0, 2: 1, 5: 2, 6: 3, 3: 4, 4: 7}
    for index, j in indeces_order.items():
        subs.append((var[index], values[j]))
    subs.append((var[1], 1))

assert len(subs) == 7*6

certain_equations = []
for equation in equations:
    if contains(equation, vars):
        certain_equations.append(equation)

assert len(certain_equations) == 65

for i, equation in enumerate(certain_equations):
    certain_equations[i] = equation.subs(subs)

In [51]:
# Нужно как можно раньше отказываться от неудачных решений.
# Идея данной оптимизации заключается в том,
# что среди уравнений исходной системы отбираются линейные.
# Для полученной СЛАУ ищется решение и проверяется невязка.

from sympy import Matrix, linear_eq_to_matrix

linear_equations = []
for equation in certain_equations:
    if equation.as_poly(vars).is_linear:
        linear_equations.append(equation)

assert len(linear_equations) == 49

M, b = linear_eq_to_matrix(linear_equations, vars)

M = np.array(M.tolist(), dtype=np.float64)
b = np.squeeze(np.array(b.tolist(), dtype=np.float64))

assert M.shape == (49, 12)
assert b.shape == (49,)

solution = np.linalg.lstsq(M, b, rcond=None)

residual = solution[1]

if len(residual) > 0 and residual[0] > 1:
    print(f"current solution must be dropped: {residual[0]}")

current solution must be dropped: 37.51413168112703


In [50]:
# Проверка того, что невязка решения, найденного Александром, действительно мало.

solved_equations = []
for equation in equations:
    if not contains(equation, vars):
        solved_equations.append(equation)

assert len(solved_equations) == 60

diff = np.array([np.float128(equation.lhs.subs(subs) - equation.rhs) for equation in solved_equations])

sum(diff**2)

1.7540564658872566938e-06

In [43]:
diff = np.array(diff)
for i, d in enumerate(diff):
    if d**2 > 0.1:
        print(i, d, solved_equations[i])

diff

39 0.3989064499910455 Eq(-a1*b1*c5 - a1*b1*c6 - a1*b5*c1 - a1*b6*c1 - a5*b1*c1 - a6*b1*c1 + 2*x1*y1*z5 + 2*x1*y1*z6 + 2*x1*y5*z1 + 2*x1*y6*z1 + 2*x5*y1*z1 + 2*x6*y1*z1, 0)


array([-5.66945962e-06,  1.36320716e-06, -1.29922468e-08,  6.49346987e-07,
        5.30607778e-06, -9.66473128e-07, -1.81518206e-05,  5.12340019e-04,
       -1.47537945e-03,  1.30816250e-04, -5.31032951e-05, -2.50360944e-03,
        7.88687227e-04, -9.99368986e-04, -6.55181055e-04,  1.20585140e-06,
       -1.17909056e-06,  2.28796744e-04,  5.19477508e-05,  4.24939972e-05,
       -1.82579087e-03,  3.93251642e-04, -1.50669517e-03, -7.57607706e-04,
        8.47972671e-04,  1.09325107e-03,  3.94777549e-04, -8.04998629e-04,
       -8.57639465e-04, -2.18303270e-04, -1.37418216e-08, -4.43703320e-06,
        7.57309860e-04,  3.80423997e-04, -3.86890661e-04,  5.08681590e-07,
       -7.04635896e-06,  0.00000000e+00,  8.00000000e-06,  3.98906450e-01,
       -6.67912640e-05,  1.98663376e-01,  2.67293704e-06, -1.98262923e-01,
        2.00379531e-01,  2.00903385e-01, -1.56641761e-03, -6.82037552e-04,
        1.43736011e-05,  9.97939220e-02,  2.41546454e-06, -9.95177436e-02,
        9.99935082e-02, -

### Statistics

In [23]:
import subprocess
from tqdm import tqdm
import time

total_iterations = 1000
progress_bar = tqdm(total=total_iterations, desc='Progress')

linear_equations_residual1000 = {}
prev_equations_residual1000 = {}

for k in range(total_iterations):
    result = subprocess.run("./kozlov/build/myprog", input="1000", capture_output=True, text=True)

    vars_order = [X, Y, Z, A, B, C]

    subs = []
    
    lines = result.stdout.split('\n')[-8:-2]
    for i, line in enumerate(lines):
        var = vars_order[i]
        values = line.replace('|', '').split()
        indeces_order = {0: 0, 2: 1, 5: 2, 6: 3, 3: 4, 4: 7}
        for index, j in indeces_order.items():
            subs.append((var[index], values[j]))
        subs.append((var[1], 1))

    assert len(subs) == 7*6
    
    certain_equations = []
    for equation in equations:
        if contains(equation, vars):
            certain_equations.append(equation)
    
    assert len(certain_equations) == 65
    
    for i, equation in enumerate(certain_equations):
        certain_equations[i] = equation.subs(subs)

    # невязка на линейной подсистеме
    linear_equations = []
    for equation in certain_equations:
        if equation.as_poly(vars).is_linear:
            linear_equations.append(equation)
    
    assert len(linear_equations) == 49
    
    M, b = linear_eq_to_matrix(linear_equations, vars)
    
    M = np.array(M.tolist(), dtype=np.float64)
    b = np.squeeze(np.array(b.tolist(), dtype=np.float64))
    
    assert M.shape == (49, 12)
    assert b.shape == (49,)
    
    solution = np.linalg.lstsq(M, b, rcond=None)
    
    residual = solution[1]
    
    if len(residual) > 0:
        linear_equations_residual1000[k] = residual[0]

    # невяхка на системе, которую решал Козлов
    solved_equations = []
    for equation in equations:
        if not contains(equation, vars):
            solved_equations.append(equation)
    
    assert len(solved_equations) == 60
    
    diff = np.array([float(equation.lhs.subs(subs) - equation.rhs) for equation in solved_equations])
    prev_equations_residual1000[k] = np.linalg.norm(diff)

    progress_bar.update(1)

progress_bar.close()

Progress: 100%|██████████| 1000/1000 [14:46:39<00:00, 53.20s/it] 


In [26]:
linear_equations_residual10

{0: 39.639109318159974,
 1: 37.519214378744216,
 2: 37.51442020752453,
 3: 37.51449315154954,
 4: 39.64570538242727,
 5: 37.51930391415728,
 6: 37.51442275785722,
 7: 37.5190437442356,
 8: 37.51914882300102,
 9: 37.51906180567331}

In [22]:
prev_equations_residual10

{0: 0.030512223071986606,
 1: 0.33637675867489203,
 2: 1.5313218809262519,
 3: 1.6138314332242372,
 4: 0.3078305164444164,
 5: 0.620229413674451,
 6: 1.5314410709529263,
 7: 0.14865288851670533,
 8: 0.3012233838678318,
 9: 0.6013501825695456}

In [27]:
import json

In [29]:
with open("linear.json", "w") as file:
    json.dump(linear_equations_residual1000, file)

In [30]:
with open("prev.json", "w") as file:
    json.dump(prev_equations_residual1000, file)

In [35]:
min(linear_equations_residual1000.values()), max(linear_equations_residual1000.values())

(37.51394209077176, 39.770922205672434)

In [6]:
sp.__file__

'/home/s02190058/.local/lib/python3.11/site-packages/sympy/__init__.py'